In [1]:
# -*- coding: utf-8 -*-
# -*- authors : Vincent Roduit, Filippo Quadri -*-
# -*- date : 2024-05-03 -*-
# -*- Last revision: 2024-05-03 -*-
# -*- python version : 3.9.18 -*-
# -*- Description: Notebook that summarize results-*-

# <center> EE - 451 Image Analysis and Pattern recognition </center>
## <center> Ecole Polytechnique Fédérale de Lausanne </center>
### <center>Coin Challenge </center>
---

In [2]:
#Import libraries
import os
import numpy as np
import torch
import importlib
import skimage as sk
from skimage.color import rgb2hsv
import matplotlib.pyplot as plt
from typing import Callable
import cv2 as cv
from skimage.morphology import closing, opening, disk, remove_small_holes, remove_small_objects
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings("ignore")

In [3]:
#Import files
from data_classes.ref_data import refCoin
from data_classes.train_data import trainCoin
from data_classes.test_data import testCoin
import constants
from visualization import *
importlib.reload(constants)
from processing.process_func import *
from models.utils import *
from processing.data_augmentation import *
%load_ext autoreload
%autoreload 2

# 1. Load different Datasets

In [4]:
ref_data = refCoin()

Loading data from pickle files


In [11]:
train_data = trainCoin(save=False)

Loading data from pickle files


In [6]:
test_data = testCoin()

Loading data from pickle files


# 2 Create Data Ready for Neural Network

In [12]:
train_data.process_images()

In [13]:
train_data.create_masked_images()

In [20]:
train_data.create_coin_images()

In [21]:
conversion_table = get_classes_conv_table()

In [22]:
coin_labels = get_coin_labels()

In [23]:
images, labels, df_images_labels = create_data_structure(train_data.coins, coin_labels, conversion_table)

In [24]:
save_coins_classified(df_images_labels, images)

In [25]:
train_images, train_labels, val_images, val_labels = create_datasets(images, labels)

# 3 Data Augmentation

In [28]:
train_images_aug, train_labels_aug = augment_set(train_images, train_labels)

In [29]:
train_images_aug, train_labels_aug = augment_blur(train_images_aug, train_labels_aug)   

# 4 Train Neural Network


In [33]:
train_dataloader, val_dataloader = create_dataloader(train_images_aug, train_labels_aug, val_images, val_labels)

## 4.1 Custom Basic CNN

In [209]:
from models.cnn import Basic_CNN
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch import nn

image_dim = images.shape[1]
num_classes = len(conversion_table)
# Define the model
cnn = Basic_CNN(image_size=image_dim, num_classes=num_classes) 

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=2, verbose=True
)
criterion = nn.BCEWithLogitsLoss()

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    train_dataloader,
    val_dataloader,
)

Epoch 1/10, Loss: 39.661583729238785, Validation Accuracy: 0.2056


KeyboardInterrupt: 

# 4.2 RESNET-50

In [ ]:
from transformers import AutoImageProcessor, ResNetForImageClassification
from torchvision import transforms
from datasets import load_metric

# Step 2: Modify the final layer
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

# Replace the classifier
num_classes = 15
model.classifier = nn.Linear(model.classifier.in_features, num_classes)

# Step 3: Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(10):  # Adjust the number of epochs as needed
    model.train()
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{10}, Loss: {loss.item()}")

# Evaluation
model.eval()
metric = load_metric("accuracy")
for images, labels in val_dataloader:
    images, labels = images.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(images).logits
    predictions = torch.argmax(outputs, dim=1)
    metric.add_batch(predictions=predictions, references=labels)

accuracy = metric.compute()
print(f"Test Accuracy: {accuracy['accuracy']:.4f}")
